In [1]:
%load_ext autoreload
%autoreload 2
import sys, codecs, json, os
import ttools
from twython import TwythonStreamer, Twython
from datetime import datetime
from time import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dateutil.parser import parse as parse_date

## Try to relate tweet with tweet_id's tweet_lang...

In [ ]:
%%time
BASENAMES = ['50M_250_1000_%s_updated.csv'%(NUM),'50M_1001_5000_%s_updated.csv'%(NUM)]
BASENAMES = ['50M_250_1000_','50M_1001_5000_']
BASEFILTERS = [(250,1001),(1001,5000)]
for BASENUM in range(len(BASENAMES)):
    print('working on subset files: %s'%(BASENAMES[BASENUM]))
    
    #load in our database of all tweets for relevant users, with new attributes we will append
    datas = []
    USECOLS = ['tweet_id','tweet_lang','date','text','user_id','user_name','user_screen','user_followers_count']
    for WAVENUM in range(1,11):
        path = '/Users/olderhorselover/USC/fall2018/csci599/teamrepo/social-swear/misc/tweets_5mAllAttributesFINAL_and_zip/tweets-5M_all_wave%s.csv'%(WAVENUM)
        iter_data = pd.read_csv(path,usecols=USECOLS,index_col=None, lineterminator='\n',iterator=True, chunksize=100000)
        data = pd.concat([chunk[(chunk['user_followers_count'] >= BASEFILTERS[BASENUM][0]) & (chunk['user_followers_count'] < BASEFILTERS[BASENUM][1])].drop(labels='user_followers_count',axis=1) for chunk in iter_data])
    #     data = data.drop(labels='user_followers_count',axis=1)
        datas.append(data)
        del iter_data
        del data
    alldatas = pd.concat(datas)
    del datas
    alldatas.shape
    print(f'Raw fullset data shape: {alldatas.shape}')
    print(f'Raw fullset data columns:\n{alldatas.columns}')
    print(f'Raw fullset data size: {alldatas.memory_usage().sum()/1000000:.2f} MB')
    
    ss = lambda x: x.replace('\r','')
    dd = lambda x: parse_date(x).strftime('%a %b %d %H:%M:%S %z %Y')
    for file in os.listdir('/Users/olderhorselover/USC/fall2018/csci599/project/processedData/50m_all_updated/'):
        if not file.startswith(BASENAMES[BASENUM]):
            continue  #do not process file
        print('working on file: %s'%file)
        path = '/Users/olderhorselover/USC/fall2018/csci599/project/processedData/50m_all_updated/%s'%(file)
        df = pd.read_csv(path,index_col=None, header=0, lineterminator='\n',converters={'text':ss,'date':dd})
        try:
            df = df.drop(labels='date1\r',axis=1)
        except:
            pass
        print(f'raw processed data shape: {df.shape}')
        print(f'raw processed data columns:\n{df.columns}')
        print(f'Raw processed data size: {df.memory_usage().sum()/1000000:.2f} MB')
        print('PERFROMING THE MERGE!')
        dfnew = df.merge(alldatas[['user_id','text','date','tweet_id','tweet_lang','user_name','user_screen']],on=['user_id','text','date'],how='left')  # new w/ new method...also drop the 'right' columns 
        print(f'orig dataframe shape: {df.shape}')
        print(f'new dataframe shape: {dfnew.shape}')
        print(f'perc diff shape: {dfnew.shape[0]/df.shape[0]:.3f}')
        naStats = dfnew.isna().sum()
        print(f'num of tweets to drop: {naStats.tweet_id}')
        print(f'new dataframe isna() stats:\n{naStats}')
        try:
            dfnew = dfnew.dropna(axis=0,how='any',subset=['tweet_id'])  #drop any tweets that didn't get a tweet_id
        except:
            pass
        print(f'FINAL dataframe shape: {dfnew.shape}')
        print('Writing dataframe to file!')
        outname = file.replace('_updated.csv','_final')
        alldata.to_csv('./PROCESSED_FINAL_11152018/%s.csv'%(outname),index=False)
        alldata.to_pickle('./PROCESSED_FINAL_11152018/%s.pkl'%(outname))
        del df
        del dfnew

    del alldatas  # no longer need/will update on next iter of BASENUM for applicable users

In [5]:
# %%time
# #longer to read in but saves time with merge [no need to create hash, faster merge on multikey]
# NUM = 1
# BASENAMES = ['50M_250_1000_%s_updated.csv'%(NUM),'50M_1001_5000_%s_updated.csv'%(NUM)]
# BASEFILTERS = [(250,1001),(1001,5000)]
# #for BASENUM in range(len(BASENAMES))
# ss = lambda x: x.replace('\r','')
# dd = lambda x: parse_date(x).strftime('%a %b %d %H:%M:%S %z %Y')
# path = '/Users/olderhorselover/USC/fall2018/csci599/project/processedData/50m_all_updated/%s'%(BASENAMES[0])
# df = pd.read_csv(path,index_col=None, header=0, lineterminator='\n',converters={'text':ss,'date':dd})
# try:
#     df = df.drop(labels='date1\r',axis=1)
# except:
#     pass

<string>:2: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 3min 15s, sys: 3.11 s, total: 3min 18s
Wall time: 3min 19s


In [38]:
file.replace('_updated.csv','_final.csv')

'50M_1001_5000_4_final.csv'

In [6]:
# print(df.shape)
# print(df.columns)
# print(f'Raw processed data size: {df.memory_usage().sum()/1000000:.2f} MB')

(1739063, 54)
Index(['user_id', 'text', 'n_retweets', 'n_favorites', 'n_user_followers',
       'n_user_following', 'n_user_posts', 'n_user_favs', 'n_user_lists',
       'user_description_text', 'user_location', 'account_age',
       'user_created_year', 'user_created_month', 'user_geo_enabled',
       'user_img_url', 'user_banner_url', 'tweet_truncated', 'tweet_source',
       'tweet_coord', 'tweet_place', 'numMentions', 'date', 'engagment',
       'is_reply', 'is_user_bot', 'is_outlier', 'is_valid', 'eggplant',
       'mfinger', 'curse', 'fuck', 'text_polarity', 'text_subjectivity',
       'vader_comp', 'vader_pos', 'vader_neu', 'vader_neg', 'processed_text',
       'text_token', 'text_token_rem_stop', 'text_len', 'n_mentions',
       'n_hashtags', 'n_links', 'n_emojis', 'swear_count', 'swear_present',
       'swear_severity', 'swear_rarity_by_percentage', 'censored_presence',
       'day_of_week', 'hour_of_day', 'per_engagement'],
      dtype='object')
Raw processed data size: 678.2

In [9]:
%%time
#load in the raw dataset [has the new attributes and all tweets]...loading ALL at once may be tough on the system...
#hmm..can we curate a new dataframe that has ALL tweets..['tweet_id','tweet_lang','date','text','user_id']
BASENUM = 0
datas = []
USECOLS = ['tweet_id','tweet_lang','date','text','user_id','user_name','user_screen','user_followers_count']
for WAVENUM in range(1,11):
#     iter_csv = pd.read_csv('file.csv', iterator=True, chunksize=1000)
#     data = pd.read_csv('/Users/olderhorselover/USC/fall2018/csci599/teamrepo/social-swear/misc/tweets_5mAllAttributesFINAL_and_zip/tweets-5M_all_wave%s.csv'%(WAVENUM),usecols=['tweet_id','tweet_lang','date','text','user_id'],index_col=None, header=0, lineterminator='\n')
    path = '/Users/olderhorselover/USC/fall2018/csci599/teamrepo/social-swear/misc/tweets_5mAllAttributesFINAL_and_zip/tweets-5M_all_wave%s.csv'%(WAVENUM)
    iter_data = pd.read_csv(path,usecols=USECOLS,index_col=None, lineterminator='\n',iterator=True, chunksize=100000)
    data = pd.concat([chunk[(chunk['user_followers_count'] >= BASEFILTERS[BASENUM][0]) & (chunk['user_followers_count'] < BASEFILTERS[BASENUM][1])].drop(labels='user_followers_count',axis=1) for chunk in iter_data])
#     data = data.drop(labels='user_followers_count',axis=1)
    datas.append(data)
    del iter_data
    del data
alldatas = pd.concat(datas)
del datas
alldatas.shape

CPU times: user 4min 13s, sys: 54.2 s, total: 5min 8s
Wall time: 5min 18s


In [10]:
print(alldatas.shape)
print(alldatas.columns)
print(f'Raw fullset data size: {alldatas.memory_usage().sum()/1000000:.2f} MB')

(19528151, 7)
Index(['tweet_id', 'date', 'tweet_lang', 'text', 'user_id', 'user_name',
       'user_screen'],
      dtype='object')
Raw fullset data size: 1249.80 MB


# THE MONEY MAKER RIGHT HERE....

## Need to pick the best/fastest/most reliable version...

In [11]:
%%time
#how='left' or 'inner' ???  ALSO...filter to 'en' tweets on our full databse beforehand? [saves memory....]
#dfnew = df.merge(alldatas[['hasher','tweet_id','tweet_lang']],on=['hasher'])  # orig
#dfnew = df.merge(alldatas[['hasher','tweet_id','tweet_lang','user_name','user_screen']],on=['hasher'])  # new w/ orig method
# dfnew = df.merge(alldatas[['user_id','text','date','tweet_id','tweet_lang','user_name','user_screen']],on=['user_id','text','date'])  
#dfnew = df.merge(alldatas[['user_id','text','date','tweet_id','tweet_lang']],on=['user_id','text','date'],how='left')  # new w/ new method...also drop the 'right' columns 
dfnew = df.merge(alldatas[['user_id','text','date','tweet_id','tweet_lang','user_name','user_screen']],on=['user_id','text','date'],how='left')  # new w/ new method...also drop the 'right' columns 
print(f'orig dataframe shape: {df.shape}')
print(f'new dataframe shape: {dfnew.shape}')
print(f'perc diff shape: {dfnew.shape[0]/df.shape[0]:.3f}')
naStats = dfnew.isna().sum()
print(f'num of tweets to drop: {naStats.tweet_id}')
print(f'new dataframe isna() stats:\n{naStats}')
dfnew = dfnew.dropna(axis=0,how='any',subset=['tweet_id'])  #drop any tweets that didn't get a tweet_id
print(dfnew.shape)

CPU times: user 37.3 s, sys: 24.4 s, total: 1min 1s
Wall time: 1min 10s


In [12]:
dfnew.isna().sum()

user_id                             0
text                                0
n_retweets                          0
n_favorites                         0
n_user_followers                    0
n_user_following                    0
n_user_posts                        0
n_user_favs                         0
n_user_lists                        0
user_description_text          141381
user_location                  424211
account_age                         0
user_created_year                   0
user_created_month                  0
user_geo_enabled                    0
user_img_url                        0
user_banner_url                 93719
tweet_truncated                     0
tweet_source                        0
tweet_coord                   1725251
tweet_place                   1626846
numMentions                         0
date                                0
engagment                           0
is_reply                            0
is_user_bot                         0
is_outlier  

In [13]:
dfnew.shape[0]/df.shape[0]

1.000010350401337

new dataframe na stats:
 user_id                             0
text                                0
n_retweets                          0
n_favorites                         0
n_user_followers                    0
n_user_following                    0
n_user_posts                        0
n_user_favs                         0
n_user_lists                        0
user_description_text          141381
user_location                  424198
account_age                         0
user_created_year                   0
user_created_month                  0
user_geo_enabled                    0
user_img_url                        0
user_banner_url                 93719
tweet_truncated                     0
tweet_source                        0
tweet_coord                   1725190
tweet_place                   1626785
numMentions                         0
date                                0
engagment                           0
is_reply                            0
is_user_bot              

In [15]:
%time du = dfnew.duplicated(subset=['user_id','text','date'])

CPU times: user 2.52 s, sys: 528 ms, total: 3.05 s
Wall time: 3.3 s


In [16]:
dfnew.columns

Index(['user_id', 'text', 'n_retweets', 'n_favorites', 'n_user_followers',
       'n_user_following', 'n_user_posts', 'n_user_favs', 'n_user_lists',
       'user_description_text', 'user_location', 'account_age',
       'user_created_year', 'user_created_month', 'user_geo_enabled',
       'user_img_url', 'user_banner_url', 'tweet_truncated', 'tweet_source',
       'tweet_coord', 'tweet_place', 'numMentions', 'date', 'engagment',
       'is_reply', 'is_user_bot', 'is_outlier', 'is_valid', 'eggplant',
       'mfinger', 'curse', 'fuck', 'text_polarity', 'text_subjectivity',
       'vader_comp', 'vader_pos', 'vader_neu', 'vader_neg', 'processed_text',
       'text_token', 'text_token_rem_stop', 'text_len', 'n_mentions',
       'n_hashtags', 'n_links', 'n_emojis', 'swear_count', 'swear_present',
       'swear_severity', 'swear_rarity_by_percentage', 'censored_presence',
       'day_of_week', 'hour_of_day', 'per_engagement', 'tweet_id',
       'tweet_lang', 'user_name', 'user_screen'],
     

In [25]:
print(f'perc diff: {dfnew.shape[0]/df.shape[0]:.3f}')

perc diff: 1.000


In [33]:
BASENAMES = ['50M_250_1000_%s_updated.csv'%(NUM),'50M_1001_5000_%s_updated.csv'%(NUM)]
BASENAMES = ['50M_250_1000_','50M_1001_5000_']
BASEFILTERS = [(250,1001),(1001,5000)]
for BASENUM in range(len(BASENAMES)):
    print(BASENAMES[BASENUM])
    for file in os.listdir('/Users/olderhorselover/USC/fall2018/csci599/project/processedData/50m_all_updated/'):
        if not file.startswith(BASENAMES[BASENUM]):
            continue  #do not process file
        print(file)

50M_250_1000_
50M_250_1000_9_updated.csv
50M_250_1000_5_updated.csv
50M_250_1000_6_updated.csv
50M_250_1000_3_updated.csv
50M_250_1000_4_updated.csv
50M_250_1000_8_updated.csv
50M_250_1000_1_updated.csv
50M_250_1000_2_updated.csv
50M_250_1000_10_updated.csv
50M_250_1000_7_updated.csv
50M_1001_5000_
50M_1001_5000_3_updated.csv
50M_1001_5000_6_updated.csv
50M_1001_5000_5_updated.csv
50M_1001_5000_7_updated.csv
50M_1001_5000_2_updated.csv
50M_1001_5000_1_updated.csv
50M_1001_5000_8_updated.csv
50M_1001_5000_4_updated.csv
